# **AI Email Automation Agent**

Managing an email inbox can be a time-consuming task, especially for busy technical managers who deal with a high volume of communications. This project demonstrates how a sophisticated AI agent can automate common email tasks: summarizing content and scheduling meetings. It aims to reduce manual effort, improve responsiveness, and enhance productivity by leveraging Large Language Models (LLMs) and external API integrations.

### Core Concepts Demonstrated:

*   **AI Agent (LangChain Agent):** A system that can dynamically reason, plan, and take actions using a set of tools to achieve a user's goal.
*   **Tool Use / Function Calling:** How an LLM can be equipped with external functions (tools) and intelligently decide when and how to use them.
*   **Conditional Logic / Intent Recognition:** The agent's ability to understand the intent behind an email (e.g., "schedule a meeting" vs. "summarize") and choose the appropriate tool.
*   **API Integration:** Interacting with real-world services like Gmail and Google Calendar.
*   **Robustness & Debugging:** Understanding and overcoming common challenges in AI agent development, such as authentication errors, LLM output formatting, and timezone handling.
*   **Custom Prompt Engineering:** Guiding the LLM's behavior with clear and specific instructions.

---

## 1. Setup: Install Required Libraries

First, we need to install all the necessary Python libraries. These include:
*   `google-api-python-client`, `google-auth-httplib2`, `google-auth-oauthlib`: For interacting with Google's Gmail and Calendar APIs.
*   `langchain`, `langchain-google-genai`, `google-generativeai`: For building our AI agent framework and connecting to Google's Gemini LLM.

**Action:** Run the following cell to install the dependencies.

In [1]:
#required libraries
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
!pip install langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 41.0 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.172.0
    Uninstalling google-api-python-client-2.172.0:
      Successfully uninstalled google-api-python-client-2.172.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might nee

---

## 2. Authentication: Connecting to Google & Gemini

This is a critical step where we authorize our Colab notebook to access your Google (Gmail & Calendar) and Google AI (Gemini) accounts.

### A. Google AI (Gemini) API Key

*   **Purpose:** Powers the "brain" of our AI agent.
*   **Action:**
    1.  Go to **Google AI Studio**: [https://aistudio.google.com/](https://aistudio.google.com/)
    2.  Click "Get API key" and create a new project/key. Copy the key.
    3.  In Colab, click the **key icon (🔑)** on the left sidebar (Secrets Manager).
    4.  Click "+ Add new secret", name it `GOOGLE_API_KEY`, and paste your Gemini API key. Toggle the switch to make it accessible.

### B. Google Workspace (Gmail & Calendar) Credentials

*   **Purpose:** Allows our agent to read your emails and manage your calendar.
*   **Action:**
    1.  Go to the **Google Cloud Console**: [https://console.cloud.google.com/](https://console.cloud.google.com/)
    2.  **Create a New Project:** If you haven't already.
    3.  **Enable APIs:** Search for and enable "Gmail API" and "Google Calendar API".
    4.  **Create OAuth Consent Screen:** Go to "APIs & Services" -> "OAuth consent screen". Choose "External", fill in basic app info, add *your own Gmail address* as a "Test user".
    5.  **Create Credentials:** Go to "APIs & Services" -> "Credentials". Click "+ CREATE CREDENTIALS" -> "OAuth client ID". Select "Desktop app", give it a name, and click "CREATE".
    6.  **Download & Rename:** Download the JSON file that appears (or from the credentials list). **Rename it to `credentials.json`**.
    7.  **Upload to Colab:** In Colab, click the **folder icon (📁)** on the left sidebar. Upload your `credentials.json` file.

        *   **Note:** If you run this cell and a `token.json` already exists with old permissions, you might need to manually delete `token.json` from your Colab files to force a fresh authorization with the new scopes.

### C. Authentication Code

This code handles the secure login process using `credentials.json` and creates service objects (`gmail_service`, `calendar_service`) to interact with Google APIs. It will prompt you to authorize in your browser the first time.


In [2]:
import os
import google.generativeai as genai
from google.colab import userdata

# --- Part 1: Configure the Gemini API ---
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)

print("✅ Gemini LLM is configured.")

# --- Part 2: Authenticate for Gmail and Calendar ---
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# permissions or scope used by our project
SCOPES = ["https://www.googleapis.com/auth/gmail.readonly", "https://www.googleapis.com/auth/calendar"] #reading emails and full access to calendar
creds = None

# The file token.json stores the user's access and refresh tokens.
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)

# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        # Start OAuth flow
        flow = InstalledAppFlow.from_client_secrets_file("/content/credentials.json", SCOPES)

        # Set redirect URI for manual auth (required in Colab)
        flow.redirect_uri = 'urn:ietf:wg:oauth:2.0:oob'

        # Generate the authorization URL
        auth_url, _ = flow.authorization_url(prompt='consent')

        print("🔗 Visit this URL to authorize the app:")
        print(auth_url)

        # Ask user for authorization code
        code = input("🔑 Paste the authorization code here: ")

        # Exchange code for token
        flow.fetch_token(code=code)
        creds = flow.credentials

    # Save the credentials for future runs
    with open("token.json", "w") as token:
        token.write(creds.to_json())

# Create service clients
gmail_service = build("gmail", "v1", credentials=creds)
calendar_service = build("calendar", "v3", credentials=creds)

print("✅ Authentication for Gmail & Calendar successful!")
print("Services are ready to use.")


✅ Gemini LLM is configured.
✅ Authentication for Gmail & Calendar successful!
Services are ready to use.


---

## 3. Defining the Agent's Tools

An AI agent's power comes from the "tools" it can use to interact with the world. We define Python functions and use LangChain's `@tool` decorator to make them available to our LLM. The docstring of each `@tool` is crucial as it tells the LLM what the tool does and what inputs it expects.

*   **`get_email_body`**: A helper function to correctly decode email content from Gmail API's base64 encoding.
*   **`get_unread_emails`**: Our tool to fetch the sender and subject lines of unread emails.
*   **`schedule_meeting`**: Our tool to create calendar events. This tool was refined to robustly handle JSON inputs and to account for timezones (a common debugging challenge!), ensuring the meeting is scheduled at the correct time in your local timezone.
*   **`summarize_email`**: Our tool to condense email content into a one-sentence summary, using the LLM itself for the summarization task.


In [3]:
import base64
import re
from datetime import datetime, timedelta, timezone
from langchain.agents import tool

def get_email_body(parts):
    """
    Recursively search for the plain text part of an email.
    """
    body = ""
    if parts:
        for part in parts:
            if part.get("mimeType") == "text/plain":

                # Decode the base64-encoded text
                data = part.get("body").get("data")
                if data:
                    body = base64.urlsafe_b64decode(data).decode("utf-8")
                return body
            elif "parts" in part:

                return get_email_body(part.get("parts"))
    return body

# --- Tool to Get Unread Emails ---
@tool
def get_unread_emails():
    """
    Fetches the subject, sender, and body of unread emails from the user's Gmail inbox.
    """
    print("---  TOOL for Checking for unread emails... ---")
    # Search for unread messages
    results = gmail_service.users().messages().list(userId="me", q="is:unread").execute()
    messages = results.get("messages", [])

    if not messages:
        return "No unread messages found."

    email_list = []
    for msg_ref in messages[:5]: #processing just 5
        msg_id = msg_ref["id"]

        #fetching message details
        msg = gmail_service.users().messages().get(userId="me", id=msg_id).execute()
        payload = msg.get("payload", {})
        headers = payload.get("headers", [])

        # Extract Subject and Sender
        subject = next((h["value"] for h in headers if h["name"] == "Subject"), "No Subject")
        sender = next((h["value"] for h in headers if h["name"] == "From"), "Unknown Sender")

        # Extract Body
        body = get_email_body(payload.get("parts"))

        if not body:
            body_data = payload.get("body", {}).get("data")
            if body_data:
                body = base64.urlsafe_b64decode(body_data).decode("utf-8")

        #email_list.append(f"Email ID: {msg_id}\nFrom: {sender}\nSubject: {subject}\n\nBody:\n{body[:1000]}")
        email_list.append(f"Email ID: {msg_id}\nFrom: {sender}\nSubject: {subject}\n")
    return "\n---\n".join(email_list)


# --- Tool to Schedule a Meeting ---
@tool
def schedule_meeting(topic: str, start_time_str: str, duration_minutes: int, attendees_emails: list[str]):
    """
    Schedules an event in the user's primary Google Calendar.
    The start_time_str must be in ISO 8601 format (e.g., '2024-05-21T15:00:00').
    attendees_emails should be a list of email addresses.
    """
    print(f"--- TOOL: Scheduling meeting about '{topic}'... ---")
    try:
        # Parse the start time and calculate the end time
        start_time = datetime.fromisoformat(start_time_str)
        # If no timezone info, assume local system timezone
        if start_time.tzinfo is None:
            start_time = start_time.astimezone()

        end_time = start_time + timedelta(minutes=duration_minutes)

        # Add the user's own email to the attendees list
        my_email = calendar_service.calendars().get(calendarId='primary').execute().get('id')
        if my_email not in attendees_emails:
            attendees_emails.append(my_email)

        # Create the event object for the API
        event = {
            'summary': topic,
            'start': {
                'dateTime': start_time.isoformat(),
                'timeZone': str(start_time.tzinfo),
            },
            'end': {
                'dateTime': end_time.isoformat(),
                'timeZone': str(start_time.tzinfo),
            },
            'attendees': [{'email': email} for email in attendees_emails],

            'reminders': {
                'useDefault': False,
                'overrides': [
                    {'method': 'popup', 'minutes': 30},
                    {'method': 'email', 'minutes': 60},
                ],
            },
        }

        # Call the Calendar API
        created_event = calendar_service.events().insert(calendarId='primary', body=event, sendUpdates='all').execute()
        return f"Success! Meeting scheduled. View it here: {created_event.get('htmlLink')}"
    except Exception as e:
        return f"Error creating event: {e}. Please check the input parameters, especially the date format."

# --- Summarize Email Tool ---
@tool
def summarize_email(email_content: str):
    """
    Analyzes the content of an email and provides a concise one-sentence summary.
    Use this for emails that are not meeting requests but contain useful information,
    like newsletters, reports, or general updates.
    Do not use this for simple notifications or ads.
    """
    print(f"--- TOOL: Summarizing email... ---")
    try:
        # We use our already-configured LLM to perform the summarization
        prompt = f"Please provide a concise, one-sentence summary of the following email content:\n\n---\n{email_content}\n---"

        # We invoke the LLM directly here for a simple task
        response = llm.invoke(prompt)

        summary = response.content
        print(f"--- TOOL: Generated summary: {summary} ---")
        return summary
    except Exception as e:
        print(f"--- TOOL ERROR: Failed to summarize. Error: {e} ---")
        return "Error: Could not summarize the email."


print("✅ Tools are defined.")
# a quick test to see if the get_unread_emails tool works
# NOTE: To test this, you should have at least one unread email in your inbox.
print("\n--- Performing a quick test of the email tool ---")
test_emails = get_unread_emails.invoke({})
print(test_emails)

✅ Tools are defined.

--- Performing a quick test of the email tool ---
---  TOOL for Checking for unread emails... ---
Email ID: 1978c3288d2db897
From: Coursera <Coursera@m.learn.coursera.org>
Subject: Learn How Data Powers Your Food Delivery | Live at 6 PM

---
Email ID: 1978c0aa5b481892
From: Unstop <noreply@emails.unstop.com>
Subject: Don’t snooze this reminder | Quizzes and/or hackathons lined up today!

---
Email ID: 1978ba8124a6a729
From: "Job Guarantee Course | Cuvette Tech" <team@cuvette.tech>
Subject: Data Analysis Placement Guarantee Program: Important Update

---
Email ID: 1978b9ae676ca71e
From: Team Videsh <hello@flyvidesh.com>
Subject: Vizuara AI Agents Bootcamp: Last day!

---
Email ID: 1978b99484afc94a
From: Unstop Insights <noreply@dare2compete.news>
Subject: Lakshita, NVIDIA is offering free tech courses—grab a seat now!



---

## 4. Building and Running the Agent

This is where the entire system comes together. We use LangChain's `create_tool_calling_agent` to link our LLM brain with the tools we've defined. The `AgentExecutor` then runs the core "Thought -> Action -> Observation" loop.

### Key Components:

*   **LLM (Gemini 1.5 Flash):** Our agent's "brain," responsible for reasoning, understanding instructions, and deciding which tools to use.
*   **Tools:** The list of functions (`get_unread_emails`, `schedule_meeting`, `summarize_email`) that the LLM can call.
*   **Custom Prompt:** We use a `ChatPromptTemplate` to give explicit instructions to the LLM. This prompt was crucial for making the agent reliably use its tools and prevent "hallucinations" where it would just talk about taking action instead of actually doing it. It also provides important context like today's date and the user's timezone.
*   **`create_tool_calling_agent`:** This is the modern LangChain construct optimized for models like Gemini that excel at generating structured tool calls (JSON). It correctly parses the LLM's output and passes arguments to our tools.
*   **`AgentExecutor`:** The runtime that executes the agent's decision-making process. `verbose=True` is enabled to show the agent's internal thoughts and tool invocations, which is invaluable for understanding and debugging.

### How to Run:

1.  Ensure you have some **unread emails** in your Gmail inbox:
    *   At least one clear meeting request (e.g., "Can we meet tomorrow at 3 PM for 45 minutes to discuss Project X? My email is person@example.com").
    *   At least one informational email (e.g., a newsletter or report).
    *   Optionally, some junk mail.
2.  **Run this cell.** Observe the detailed verbose output as the agent processes each email, decides whether to schedule, summarize, or ignore, and finally reports its actions.


In [10]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.prompts import ChatPromptTemplate
from google.colab import userdata

# Retrieve the API key
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')

# 1. Define the LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    google_api_key=GEMINI_API_KEY
)
print("🧠 LLM (Gemini 2.5 Flash) is ready.")

# 2. Define Tools
tools = [get_unread_emails, schedule_meeting, summarize_email]
print(f"🧰 Agent has access to {len(tools)} tools: {[t.name for t in tools]}")

# 3. Prompt Template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. You have access to tools."),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
        ("system", "If you have used a tool and have an observation, you MUST use that information to answer the user's request. Do not make up results."),
    ]
)
print("📜 Custom, more forceful prompt is loaded.")

# 4. Create the Agent itself
agent = create_tool_calling_agent(llm, tools, prompt)
print("🤖 Tool-calling agent logic is created.")

# 5. Create the Agent Executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
print("🚀 Agent Executor is ready to run.")

print("\n--- Starting the Agent ---")
task = """
First, check my unread emails.
Then, for each email, decide on ONE of the following actions:
1. If the mail has details like name, email(or use the sender email), date, time, duration or a meeting request, use the 'schedule_meeting' tool to schedule a meeting.
2. If the email is a newsletter, report, or a long informational update, use the 'summarize_email' tool.
3. If the email is an advertisement, a simple notification, or junk, just ignore it.

Finally, report a summary of all the actions you have taken.
For context, today's date is May 22, 2024, and my local timezone is IST (India Standard Time, which is UTC+05:30).
For example, 3 PM IST should be 'YYYY-MM-DD-T15:00:00+05:30'.
"""

# Let's invoke the agent!
result = agent_executor.invoke({
    "input": task,
    "chat_history": []
})

print("\n--- Agent Finished ---")
print("\n✅ Final Answer from the Agent:")
print(result['output'])

🧠 LLM (Gemini 1.5 Flash) is ready.
🧰 Agent has access to 3 tools: ['get_unread_emails', 'schedule_meeting', 'summarize_email']
📜 Custom, more forceful prompt is loaded.
🤖 Tool-calling agent logic is created.
🚀 Agent Executor is ready to run.

--- Starting the Agent ---


> Entering new AgentExecutor chain...

Invoking: `get_unread_emails` with `{}`


---  TOOL for Checking for unread emails... ---
Email ID: 1978c85233d53a31
From: Lakshita Soni <theshaitaan1@gmail.com>
Subject: project sync

---
Email ID: 1978c3288d2db897
From: Coursera <Coursera@m.learn.coursera.org>
Subject: Learn How Data Powers Your Food Delivery | Live at 6 PM

---
Email ID: 1978c0aa5b481892
From: Unstop <noreply@emails.unstop.com>
Subject: Don’t snooze this reminder | Quizzes and/or hackathons lined up today!

---
Email ID: 1978ba8124a6a729
From: "Job Guarantee Course | Cuvette Tech" <team@cuvette.tech>
Subject: Data Analysis Placement Guarantee Program: Important Update

---
Email ID: 1978b9ae676ca71e
From: Team 